In [2]:
import os
import kaggle as kg
import os
import pandas as pd
import pathlib
import numpy as np
from keras.models import Model
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Lambda ,Dense ,Flatten ,Dropout
from keras.preprocessing.image import load_img, img_to_array

2024-07-23 19:10:56.413167: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-23 19:10:56.501249: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 19:10:57.100471: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 19:10:57.104176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 19:10:59.818661: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
def train_test_df(path):
    image = list()
    label = list()
    image2 = list()
    label2 = list()

    for sub_dir in pathlib.Path(path).glob("*"):
        for i,img_path in enumerate(pathlib.Path(sub_dir).glob("*.jpeg")):
            if (i<=59):
                image.append(str(img_path))
                label.append(str(img_path).split("_")[2])
            else:
                image2.append(str(img_path))
                label2.append(str(img_path).split("_")[2])
    return pd.DataFrame(data={"img_path":image,"label":label}),pd.DataFrame(data={"img_path":image2,"label":label2})

In [4]:
dataset_url = "dataset/asl_dataset"

In [5]:
train_df,test_df = train_test_df(dataset_url)

In [6]:
train_df

,img_path,label
0,dataset/asl_dataset/6/hand5_6_dif_seg_1_croppe...,6
1,dataset/asl_dataset/6/hand2_6_left_seg_2_cropp...,6
2,dataset/asl_dataset/6/hand3_6_dif_seg_5_croppe...,6
3,dataset/asl_dataset/6/hand5_6_bot_seg_1_croppe...,6
4,dataset/asl_dataset/6/hand4_6_bot_seg_1_croppe...,6
...,...,...
2155,dataset/asl_dataset/p/hand3_p_dif_seg_2_croppe...,p
2156,dataset/asl_dataset/p/hand3_p_dif_seg_3_croppe...,p
2157,dataset/asl_dataset/p/hand1_p_dif_seg_4_croppe...,p
2158,dataset/asl_dataset/p/hand3_p_dif_seg_1_croppe...,p


In [7]:
test_df

,img_path,label
0,dataset/asl_dataset/6/hand1_6_dif_seg_3_croppe...,6
1,dataset/asl_dataset/6/hand2_6_left_seg_1_cropp...,6
2,dataset/asl_dataset/6/hand2_6_top_seg_3_croppe...,6
3,dataset/asl_dataset/6/hand5_6_bot_seg_3_croppe...,6
4,dataset/asl_dataset/6/hand1_6_bot_seg_5_croppe...,6
...,...,...
350,dataset/asl_dataset/p/hand2_p_left_seg_2_cropp...,p
351,dataset/asl_dataset/p/hand1_p_dif_seg_2_croppe...,p
352,dataset/asl_dataset/p/hand5_p_bot_seg_1_croppe...,p
353,dataset/asl_dataset/p/hand2_p_dif_seg_3_croppe...,p


In [8]:
character_2_int = dict(zip(train_df["label"].unique(),range(len(train_df["label"].unique()))))

In [9]:
character_2_int

{'6': 0,
 '4': 1,
 'm': 2,
 'f': 3,
 'j': 4,
 'h': 5,
 'x': 6,
 'n': 7,
 '7': 8,
 'l': 9,
 '5': 10,
 'a': 11,
 '3': 12,
 '9': 13,
 '2': 14,
 'y': 15,
 'r': 16,
 's': 17,
 'i': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'd': 22,
 'g': 23,
 'b': 24,
 'e': 25,
 'o': 26,
 '8': 27,
 'w': 28,
 'q': 29,
 '0': 30,
 'k': 31,
 '1': 32,
 'z': 33,
 'c': 34,
 'p': 35}

In [10]:
train_df["label"].replace(to_replace=character_2_int.keys(),value=character_2_int.values(),inplace=True)

test_df["label"].replace(to_replace=character_2_int.keys(),value=character_2_int.values(),inplace=True)

/tmp/ipykernel_26108/2319185013.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["label"].replace(to_replace=character_2_int.keys(),value=character_2_int.values(),inplace=True)
/tmp/ipykernel_26108/2319185013.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df["label"].replace(t

In [11]:
train_df

,img_path,label
0,dataset/asl_dataset/6/hand5_6_dif_seg_1_croppe...,0
1,dataset/asl_dataset/6/hand2_6_left_seg_2_cropp...,0
2,dataset/asl_dataset/6/hand3_6_dif_seg_5_croppe...,0
3,dataset/asl_dataset/6/hand5_6_bot_seg_1_croppe...,0
4,dataset/asl_dataset/6/hand4_6_bot_seg_1_croppe...,0
...,...,...
2155,dataset/asl_dataset/p/hand3_p_dif_seg_2_croppe...,35
2156,dataset/asl_dataset/p/hand3_p_dif_seg_3_croppe...,35
2157,dataset/asl_dataset/p/hand1_p_dif_seg_4_croppe...,35
2158,dataset/asl_dataset/p/hand3_p_dif_seg_1_croppe...,35


In [12]:
test_df

,img_path,label
0,dataset/asl_dataset/6/hand1_6_dif_seg_3_croppe...,0
1,dataset/asl_dataset/6/hand2_6_left_seg_1_cropp...,0
2,dataset/asl_dataset/6/hand2_6_top_seg_3_croppe...,0
3,dataset/asl_dataset/6/hand5_6_bot_seg_3_croppe...,0
4,dataset/asl_dataset/6/hand1_6_bot_seg_5_croppe...,0
...,...,...
350,dataset/asl_dataset/p/hand2_p_left_seg_2_cropp...,35
351,dataset/asl_dataset/p/hand1_p_dif_seg_2_croppe...,35
352,dataset/asl_dataset/p/hand5_p_bot_seg_1_croppe...,35
353,dataset/asl_dataset/p/hand2_p_dif_seg_3_croppe...,35


In [13]:
y_true_train = to_categorical(y=train_df["label"],num_classes=36)
y_true_test = to_categorical(y=test_df["label"],num_classes=36)

In [14]:
y_true_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [15]:
y_true_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [16]:
def data_generator(data_df, y_true, mb_size):
    num_batches = data_df.shape[0] // mb_size

    while True:
        for batch_id in range(num_batches):
            x_mb = list()
            batch_start = batch_id * mb_size
            batch_end = batch_start + mb_size

            generated_img_paths = data_df.iloc[batch_start:batch_end]["img_path"].to_list()
            for img_path in generated_img_paths:
                img = load_img(img_path, target_size=(64, 64))
                img_np_arr = img_to_array(img)
                x_mb.append(img_np_arr)

            x_mb = np.array(x_mb)
            y_true_mb = y_true[batch_start:batch_end]

            yield x_mb, y_true_mb

In [17]:
generator_train = data_generator(train_df, y_true_train, 64)
generator_test = data_generator(test_df, y_true_test, 64)

In [19]:
classifier_vgg16 = VGG16(input_shape= (64,64,3),include_top=False,weights='imagenet')

In [20]:
for layer in classifier_vgg16.layers:
    layer.trainable = False

In [21]:
classifier1 = classifier_vgg16.output
classifier1 = Flatten()(classifier1)
classifier1 = Dense(units=256, activation='relu')(classifier1)
classifier1 = Dropout(0.6)(classifier1)
classifier1 = Dense(units=36, activation='softmax')(classifier1)

model = Model(inputs = classifier_vgg16.input , outputs = classifier1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [23]:
history = model.fit(generator_train,
                    steps_per_epoch=train_df.shape[0] // 64,
                    epochs=15,
                    validation_data=generator_test,
                    validation_steps=test_df.shape[0] // 64)

Epoch 1/15
33/33 [==============================] - 32s 942ms/step - loss: 31.3374 - accuracy: 0.0109 - val_loss: 2.5932 - val_accuracy: 0.2969
Epoch 2/15
33/33 [==============================] - 31s 942ms/step - loss: 3.5741 - accuracy: 0.1065 - val_loss: 2.6205 - val_accuracy: 0.3406
Epoch 3/15
33/33 [==============================] - 39s 1s/step - loss: 2.9930 - accuracy: 0.1899 - val_loss: 2.1531 - val_accuracy: 0.4531
Epoch 4/15
33/33 [==============================] - 40s 1s/step - loss: 2.7373 - accuracy: 0.2509 - val_loss: 1.8448 - val_accuracy: 0.5375
Epoch 5/15
33/33 [==============================] - 36s 1s/step - loss: 2.4016 - accuracy: 0.3106 - val_loss: 1.4652 - val_accuracy: 0.6438
Epoch 6/15
33/33 [==============================] - 40s 1s/step - loss: 2.2099 - accuracy: 0.3504 - val_loss: 1.1630 - val_accuracy: 0.7000
Epoch 7/15
33/33 [==============================] - 35s 1s/step - loss: 2.1653 - accuracy: 0.3707 - val_loss: 1.1253 - val_accuracy: 0.7063
Epoch 8/15
33